# Python functies voor Odata4
Functies voor ophalen, inspecteren en samenvoegen van data van CBS

## Meer info

- https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/snelstartgids-odata-v4
- https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/metadata-odata-v4
- Ook code voor R beschikbaar.

# Definieren functies

In [1]:
import pandas as pd
import requests
import re

In [2]:
import pandas as pd
import requests

def get_odata(target_url):
    
    """"
    De functie gebruikt een API genaamd OData om data van het CBS op te halen.
    De data wordt in stukken opgehaald en in een pandas dataframe gezet.
    De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
    De code van de tabel die je zoekt vindt je via Statline.
    Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
    """
    
    data = pd.DataFrame()
    while target_url:
        r = requests.get(target_url).json()
        data = data.append(pd.DataFrame(r['value']))
        
        if '@odata.nextLink' in r:
            target_url = r['@odata.nextLink']
        else:
            target_url = None
            
    return data

De functie gebruikt een API genaamd OData om data van het CBS op te halen.
De data wordt in stukken opgehaald en in een pandas dataframe gezet.
De URL moet er zo uitzien: "https://odata4.cbs.nl/CBS/83765NED"
De code van de tabel die je zoekt vindt je via Statline.
Ga naar de data in Statline op de website van het CBS en kijk naar de URL om de code te vinden.
Deze API is hierarchisch. 

In [3]:
def get_observations(table_url, url_filter = ""):
    
    """Haal de tabel met metingen op. Filteren om minder of specifiekere data op te vragen is mogelijk as volgt:
    url_filter = ?$filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'
    Door de filteren op deze kolommen kun je een plaats (land, gemeente, wijk of buurt) en dan een soort meting kiezen.
    De website van het CBS vermeldt niet hoe je op meerdere waarden in één kolom filtert. 
    Die mogelijkheid bestaat waarschijnlijk wel. Je hoeft niet beide kolommen te gebruiken voor het filter.
    """
    
    if url_filter == "":
        target_url = table_url + "/Observations"
    elif "?$filter=" in url_filter:
        target_url = table_url + "/Observations" + url_filter
    else:
        print("WAARSCHUWING! FILTER NIET GOED GEFORMATTEERD. VERGELIJK MET VOORBEELD OF GA NAAR ")
        print("\n https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4")
        #return None
        pass
    
    data = get_odata(target_url)
    print(data.head())
    
    return(data)

url_voorbeeld = "https://odata4.cbs.nl/CBS/83765NED"

# Wat is de structuur van de data?

In [4]:
data_structuur = get_odata(url_voorbeeld)
print(data_structuur)
data_Mcodes = get_odata(url_voorbeeld+"/MeasureCodes")
print(data_Mcodes.columns)

        kind                   name                    url
0  EntitySet          MeasureGroups          MeasureGroups
1  EntitySet           MeasureCodes           MeasureCodes
2  EntitySet             Dimensions             Dimensions
3  EntitySet  WijkenEnBuurtenGroups  WijkenEnBuurtenGroups
4  EntitySet   WijkenEnBuurtenCodes   WijkenEnBuurtenCodes
5  EntitySet           Observations           Observations
6  Singleton             Properties             Properties
Index(['DataType', 'Decimals', 'Description', 'Format', 'Identifier', 'Index',
       'MeasureGroupId', 'PresentationType', 'Title', 'Unit'],
      dtype='object')


In [6]:
#data_Mcodes.head(10)
#data_Mcodes['Title'].values
data_Mcodes["Identifier"]

0        T001036
1           3000
2           4000
3          10680
4          53050
5          53310
6          53715
7          80200
8           1010
9           1020
10          1080
11          1050
12       2012655
13     2012657_2
14       H008673
15       H007119
16       H008751
17       H008766
18       A008187
19     M000173_1
20     M000173_2
21     M000179_1
22     M000179_2
23     1050010_2
24       1050015
25       1016040
26       1016030
27       M000114
28       M000100
29       M000297
         ...    
73     M000200_2
74        301000
75        300003
76        300005
77        383105
78        300009
79        300010
80        300014
81     A018943_2
82         40001
83         72003
84       A019276
85       M002179
86       M000368
87     A018943_5
88       A018944
89       D000028
90       D000025
91       D000029
92       D000045
93       D000263
94     T001455_2
95       A047044
96       A047040
97       M000217
98        ST0001
99        ST0003
100      CRI11

In [20]:
# het voorbeeld:
#table_url = "https://odata4.cbs.nl/CBS/83765NED"

#target_url = table_url + "/Observations"
#data = get_odata(target_url)
#print(data.head())

   Id  Measure       Value ValueAttribute WijkenEnBuurten
0   0  T001036  17081507.0           None            NL00
1   1     3000   8475102.0           None            NL00
2   2     4000   8606405.0           None            NL00
3   3    10680   2781768.0           None            NL00
4   4    53050   2101648.0           None            NL00


In [3]:
#tabel 2019
#lastyear_url = "https://odata4.cbs.nl/CBS/84583NED"

,name,url
0,TableInfos,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
1,UntypedDataSet,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
2,TypedDataSet,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
3,DataProperties,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
4,CategoryGroups,https://opendata.cbs.nl/oDataAPI/OData/84583NE...
5,WijkenEnBuurten,https://opendata.cbs.nl/oDataAPI/OData/84583NE...


## Filteren van query
Het filteren van de data maakt het downloaden sneller.
Het filteren van 'Observations' data kan door code van dit format achter de url te plakken:

**?$filter=WijkenEnBuurten eq 'GM0363' and Measure eq 'T001036'**

De code uit de kolom WijkenEnBuurten kun je vinden met 

**get_odata(table_url + "/WijkenEnBuurtenCodes")**

De 'Title' kolom van deze tabel bevat de namen van wijken, zodat je kan zoeken met str.find, <>.str.contains of Regex.
Zoals wel vaken met tektskolommen moet je dan vertrouwen op de volledigheid en consistentie.
Achteraf controleren of je alle wijken hebt is dus wel aangeraden.
De kolom WijkenenBuurten bevat zowel landen, gemeenten, wijken als buurten.
Aan het voorvoegsel van twee letters kun je zien met welke soort regio je te maken hebt.

Zie https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/filters-odata-v4 voor meer uitleg.

In [23]:
#table_test = get_observations(url_voorbeeld, url_filter="?$filter=WijkenEnBuurten eq 'GM0363'")
#table_test = get_observations(url_voorbeeld)

   Id  Measure       Value ValueAttribute WijkenEnBuurten
0   0  T001036  17081507.0           None            NL00
1   1     3000   8475102.0           None            NL00
2   2     4000   8606405.0           None            NL00
3   3    10680   2781768.0           None            NL00
4   4    53050   2101648.0           None            NL00


In [8]:
print(table_test.columns)
print(table_test.head(30))
# Kolom ValueAttribute heeft geen waarden

Index(['Id', 'Measure', 'Value', 'ValueAttribute', 'WijkenEnBuurten'], dtype='object')
       Id    Measure     Value ValueAttribute WijkenEnBuurten
0   88579    CRI3000       9.0           None          GM0363
1   88578    CRI2000       6.0           None          GM0363
2   88577    CRI1100       6.0           None          GM0363
3   88576     ST0003    6004.0           None          GM0363
4   88575     ST0001       1.0           None          GM0363
5   88573    A047040    5416.0           None          GM0363
6   88572    A047044   16533.0           None          GM0363
7   88571  T001455_2   21949.0           None          GM0363
8   88570    D000263      28.5           None          GM0363
9   88569    D000045       0.5           None          GM0363
10  88568    D000029       0.4           None          GM0363
11  88567    D000025       0.5           None          GM0363
12  88566    D000028       0.5           None          GM0363
13  88565    A018944   18750.0           None

In [9]:
# Deze tabel bevat alle gemeente, maar GEEN info over wijken en buurten
data_gemeenten = get_odata(url_voorbeeld+"/WijkenEnBuurtenGroups")
print(data_gemeenten.size)
print(data_gemeenten.columns)
print(data_gemeenten.head(10))
# ParentId geeft voor wijken de buurt aan, voor wijken de plaatsnaam, voorplaatsnamen de gemeente etc.

1960
Index(['Description', 'Id', 'Index', 'ParentId', 'Title'], dtype='object')
  Description      Id  Index ParentId                           Title
0        None    WBGM      0     None  Wijken en buurten per gemeente
1        None  GM1680      1     WBGM                     Aa en Hunze
2        None  GM0738      2     WBGM                         Aalburg
3        None  GM0358      3     WBGM                        Aalsmeer
4        None  GM0197      4     WBGM                          Aalten
5        None  GM0059      5     WBGM                   Achtkarspelen
6        None  GM0482      6     WBGM                    Alblasserdam
7        None  GM0613      7     WBGM                   Albrandswaard
8        None  GM0361      8     WBGM                         Alkmaar
9        None  GM0141      9     WBGM                          Almelo


In [10]:
filter_gemeente = data_gemeenten['Title'] == 'Oss'
print(data_gemeenten[filter_gemeente])
selectie_gem = data_gemeenten[filter_gemeente]['Id'].values[0]
selectie_gem

    Description      Id  Index ParentId Title
254        None  GM0828    254     WBGM   Oss


'GM0828'

In [11]:
# Deze tabel bevat codes van wijken en buurten en toont bij welke gemeente ze horen. De gemeenten staan er ook in.
data_geocodes = get_odata(url_voorbeeld+"/WijkenEnBuurtenCodes")
print(data_geocodes.size)
print(data_geocodes.columns)
print(data_geocodes.head(10))
# Ik denk dat DimensionGroupId te maken heeft met hierarchische indeling maar niet hetzelfde is als parentId.
# DetailRegionCode is hetzelfde als Identifier

100002
Index(['Description', 'DetailRegionCode', 'DimensionGroupId', 'Identifier',
       'Index', 'Title'],
      dtype='object')
  Description DetailRegionCode DimensionGroupId  Identifier  Index  \
0                         None               NL        NL00      1   
1                       GM1680               GM      GM1680      2   
2                     WK168000           GM1680    WK168000      3   
3                   BU16800000           GM1680  BU16800000      4   
4                   BU16800009           GM1680  BU16800009      5   
5                     WK168001           GM1680    WK168001      6   
6                   BU16800100           GM1680  BU16800100      7   
7                   BU16800109           GM1680  BU16800109      8   
8                     WK168002           GM1680    WK168002      9   
9                   BU16800200           GM1680  BU16800200     10   

                     Title  
0                Nederland  
1              Aa en Hunze  
2          

In [12]:
# Gebruik van Regular Expressions sterk aanbevolen om verschil met hele woorden te zien,
# Maakt implementatie wel iets ingewikkelder.
#filter_groepen_BU = data_gemeenten['Id'].str.contains("GM")
filter_geocodes_BU = data_geocodes['Identifier'].str.contains("BU")
filter_geocodes_WK = data_geocodes['Identifier'].str.contains("WK")

filter_BUWKopgem = data_geocodes['DimensionGroupId'] == selectie_gem

In [13]:
selectie_BU = data_geocodes[filter_geocodes_BU & filter_BUWKopgem]
selectie_WK = data_geocodes[filter_geocodes_WK & filter_BUWKopgem]

# Selecteren features en samenvoegen data

In [14]:
def feature_select(text_file):
    features = open(text_file,"r")
    features = features.readlines()
    features2 = ""
    for line in features:
        features2 += line
    # Verwijder \n, scheid op , en verwijder spaties aan begin
    features2 = re.sub("\n", "", features2)
    features2 = re.split(",", features2)
    features = []
    for feat in features2:
        features.append(re.sub("^ ","", feat))
        
    return features

features = feature_select("features_test1.txt")
#features

In [15]:
url_test = "?$filter=WijkenEnBuurten eq '"+selectie_gem+"'"
# Alleen op deze manier kan je een ID uit de geografische tabel in het url-filter plaatsen. Ook de spaties tellen.)
data_observations = get_observations(url_voorbeeld, url_filter=url_test)
data_observations.head()

        Id  Measure   Value ValueAttribute WijkenEnBuurten
0  1169358  CRI3000     4.0           None          GM0828
1  1169357  CRI2000     5.0           None          GM0828
2  1169356  CRI1100     3.0           None          GM0828
3  1169355   ST0003  1360.0           None          GM0828
4  1169354   ST0001     3.0           None          GM0828


,Id,Measure,Value,ValueAttribute,WijkenEnBuurten
0,1169358,CRI3000,4.0,None,GM0828
1,1169357,CRI2000,5.0,None,GM0828
2,1169356,CRI1100,3.0,None,GM0828
3,1169355,ST0003,1360.0,None,GM0828
4,1169354,ST0001,3.0,None,GM0828


In [17]:
data_observations["Measure"]

0        CRI3000
1        CRI2000
2        CRI1100
3         ST0003
4         ST0001
5        A047040
6        A047044
7      T001455_2
8        D000263
9        D000045
10       D000029
11       D000025
12       D000028
13       A018944
14     A018943_5
15       M000368
16       M002179
17       A019276
18         72003
19         40001
20     A018943_2
21        300014
22        300010
23        300009
24        383105
25        300005
26        300003
27        301000
28     M000200_2
29       D000193
         ...    
71       M000297
72       M000100
73       M000114
74       1016030
75       1016040
76       1050015
77     1050010_2
78     M000179_2
79     M000179_1
80     M000173_2
81     M000173_1
82       A008187
83       H008766
84       H008751
85       H007119
86       H008673
87     2012657_2
88       2012655
89          1050
90          1080
91          1020
92          1010
93         80200
94         53715
95         53310
96         53050
97         10680
98          40

In [20]:
test = data_observations.pivot(index="Id", columns = 'Measure', values="Value")
print(test.columns, len(test.columns))
# Dataset heeft nu kolom voor elke measure maar rijen nog niet samengevoegd.

Index(['1010', '1014800_1', '1014800_2', '1014800_3', '1014850_2', '1014850_3',
       '1014850_4', '1016030', '1016040', '1020',
       ...
       'ZW10320_2', 'ZW10340', 'ZW25805_1', 'ZW25805_2', 'ZW25806_1',
       'ZW25806_2', 'ZW25807', 'ZW25808', 'ZW25810_1', 'ZW25810_2'],
      dtype='object', name='Measure', length=101) 101


In [31]:
test2=table_test.pivot(index="Id", columns="Measure", values="Value") # Zelfs voor hele dataset snel te doen

In [32]:
test2 = test2.merge(table_test[["Id","WijkenEnBuurten"]], how="left", left_index=True, right_on="Id").drop("Id", axis=1)
test2.head(20)

,1010,1014800_1,1014800_2,1014800_3,1014850_2,1014850_3,1014850_4,1016030,1016040,1020,...,ZW10340,ZW25805_1,ZW25805_2,ZW25806_1,ZW25806_2,ZW25807,ZW25808,ZW25810_1,ZW25810_2,WijkenEnBuurten
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
8,8201061.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6718468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL00


In [33]:
test2.groupby("WijkenEnBuurten").first()
# Deze tabel heeft het gewenste format. 

,1010,1014800_1,1014800_2,1014800_3,1014850_2,1014850_3,1014850_4,1016030,1016040,1020,...,ZW10320_2,ZW10340,ZW25805_1,ZW25805_2,ZW25806_1,ZW25806_2,ZW25807,ZW25808,ZW25810_1,ZW25810_2
WijkenEnBuurten,,,,,,,,,,,,,,,,,,,,,
BU00030000,935.0,39.0,2770.0,1800.0,59.0,1870.0,1090.0,235.0,365.0,850.0,...,2240.0,48.0,2270.0,1320.0,2620.0,1660.0,17.0,83.0,1720.0,960.0
BU00030001,1320.0,69.0,2870.0,1600.0,31.0,2390.0,1220.0,480.0,500.0,1435.0,...,1750.0,8.0,2490.0,1280.0,2740.0,1420.0,8.0,92.0,2050.0,920.0
BU00030002,2530.0,40.0,2750.0,1550.0,59.0,2180.0,1270.0,920.0,855.0,2430.0,...,2040.0,29.0,2660.0,1400.0,2630.0,1550.0,16.0,84.0,1790.0,970.0
BU00030007,160.0,95.0,3210.0,1690.0,5.0,NaN,NaN,55.0,35.0,140.0,...,1870.0,2.0,NaN,NaN,NaN,NaN,54.0,46.0,NaN,NaN
BU00030008,40.0,90.0,3690.0,1830.0,10.0,NaN,NaN,15.0,20.0,55.0,...,1930.0,0.0,NaN,NaN,NaN,NaN,7.0,93.0,NaN,NaN
BU00030009,80.0,78.0,3260.0,2330.0,20.0,1800.0,1550.0,25.0,20.0,80.0,...,2320.0,5.0,NaN,NaN,NaN,NaN,8.0,92.0,NaN,NaN
BU00050000,3380.0,64.0,2780.0,1610.0,35.0,2050.0,1260.0,1175.0,1105.0,3485.0,...,1940.0,17.0,2570.0,1400.0,2570.0,1530.0,13.0,87.0,1670.0,880.0
BU00050001,385.0,84.0,3170.0,1710.0,16.0,1990.0,1220.0,130.0,110.0,375.0,...,2150.0,0.0,2050.0,910.0,2100.0,1020.0,53.0,47.0,NaN,NaN
BU00050002,380.0,74.0,2910.0,1960.0,26.0,2150.0,1270.0,120.0,155.0,400.0,...,2150.0,3.0,2190.0,1190.0,2390.0,1510.0,3.0,97.0,NaN,NaN


# To do: 
- Stel functie op die de pivot, merge en groupby doet.
- Maak dictionary om Measure-codes makkelijk te vertalen.
- Maak een functie die de ongewenste features weghaalt.
- Controleer op NaN's. (Kan SciKit niet goed tegen)

Dan zouden alle gewenste functies klaar moeten zijn en kan het notebook worden opgeschoont of omgezet naar .py
De hierarchische indeling van locaties moet nog wel toegevoegd worden maar omdat daar andere bestanden voor nodig zijn
is het netjes om in een ander notebook verder te gaan.

Kies de kolommen die je nodig hebt en filter op rijen en plaatsen.
Gebruik pivot functie in pandas na het filteren?